<a href="https://colab.research.google.com/github/ShirFah/Big_DataProject/blob/main/Copy_of_Netflix_popular_genres_by_season.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!git clone https://github.com/ShirFah/Big_DataProject.git

Cloning into 'Big_DataProject'...


In [21]:
!pip install pandas numpy matplotlib datetime sklearn scipy seaborn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import datetime
import math


In [23]:
df_genres = pd.read_csv("titles.csv" , encoding='latin-1')
pd.set_option('display.max_columns',15)

In [24]:
def find_genres_for_row(row):
    list_genres= df_genres.loc[df_genres['title'] == row['show_title']].values.tolist()
    if len(list_genres) >0:
        return list_genres[0][7]
    return np.nan
def add_season(row):
    date = row['week']
    month = pd.to_datetime(date).month
    return month%12 // 3 + 1



In [25]:
def prepare_data(weekly_popularity_path):
    df_pop = pd.read_csv(weekly_popularity_path)
    df_pop = df_pop.drop(['category'],axis=1) # Removing category column
    df_pop['category'] = df_pop.apply(lambda row : find_genres_for_row(row),axis=1) # match category for title
    df_pop = df_pop.dropna()# Delete rows where Category is None i.e couldnt find category for specific title
    df_pop['season'] = df_pop.apply(lambda row : add_season(row),axis=1) # add yearly season 
    return df_pop

In [26]:
def create_season_df(df):
  season_list=[] # 0 - Winter 1 - Spring 2 - Summer 3 - Autumn
  for i in range(4):
    season_list.append(df.loc[(df['season'] == (i+1))])
  category_list = df['category'].unique().tolist()
  category_lst=[]
  category_set=set()
  for lt in category_list:
    category_lst.append(lt.replace('\'','').replace('[','').replace(']','').replace(' ','').split(','))
  for item in category_lst:
    for category in item:
      category_set.add(category)
  dict_list = []
  del season_list[1]
  for i in range(3):
    dict_list.append(dict.fromkeys(category_set,0))
  count =0
  for df in season_list:
    for index,row in df.iterrows():
      lt = row['category']
      category_list_row = lt.replace('\'','').replace('[','').replace(']','').replace(' ','').split(',')
      for category in category_list_row:
        dict_list[count][category] += 1+pow(2,1/row['weekly_rank'])
    count+=1
  count=0
  for dict_pop in dict_list:
    plt.xticks(rotation=90,fontsize=12)
    plt.ylabel("Popularity")
    x=dict_pop.keys()
    y=dict_pop.values()
    plt.scatter(x,y)
    if count==0:
      plt.title("Winter")
    elif count==1:
      plt.title("Summer")
    else:
      plt.title("Autumn")
    plt.tight_layout()
    plt.savefig("fig"+str(count)+".png",dpi=300)
    plt.clf()
    count+=1




In [27]:
weekly_popularity_df = prepare_data("all-weeks-global.csv") 
season_list = create_season_df(weekly_popularity_df)

<Figure size 432x288 with 0 Axes>